In [1]:
import soundfile as sf
import librosa
import ast
import numpy as np
import pandas as pd

In [2]:

with open('my_data/float_data_from_device.txt', 'r') as f:
   a = np.array([ast.literal_eval(line) for line in f][0], dtype='float32')

sf.write('my_data/float_data_from_device.wav', a, 44100)

pd.DataFrame(a).describe()


,0
count,220500.000000
mean,0.000008
std,0.003087
min,-0.035370
25%,-0.001312
50%,0.000031
75%,0.001343
max,0.029633


In [3]:
DEFAULT_FRAME_SIZE = 512
zcr_feat = librosa.feature.zero_crossing_rate(y=a, hop_length=DEFAULT_FRAME_SIZE)

print(len(zcr_feat[0]), len(a))

431 220500


In [4]:
sf.available_formats()

{'AIFF': 'AIFF (Apple/SGI)',
 'AU': 'AU (Sun/NeXT)',
 'AVR': 'AVR (Audio Visual Research)',
 'CAF': 'CAF (Apple Core Audio File)',
 'FLAC': 'FLAC (Free Lossless Audio Codec)',
 'HTK': 'HTK (HMM Tool Kit)',
 'SVX': 'IFF (Amiga IFF/SVX8/SV16)',
 'MAT4': 'MAT4 (GNU Octave 2.0 / Matlab 4.2)',
 'MAT5': 'MAT5 (GNU Octave 2.1 / Matlab 5.0)',
 'MPC2K': 'MPC (Akai MPC 2k)',
 'OGG': 'OGG (OGG Container format)',
 'PAF': 'PAF (Ensoniq PARIS)',
 'PVF': 'PVF (Portable Voice Format)',
 'RAW': 'RAW (header-less)',
 'RF64': 'RF64 (RIFF 64)',
 'SD2': 'SD2 (Sound Designer II)',
 'SDS': 'SDS (Midi Sample Dump Standard)',
 'IRCAM': 'SF (Berkeley/IRCAM/CARL)',
 'VOC': 'VOC (Creative Labs)',
 'W64': 'W64 (SoundFoundry WAVE 64)',
 'WAV': 'WAV (Microsoft)',
 'NIST': 'WAV (NIST Sphere)',
 'WAVEX': 'WAVEX (Microsoft)',
 'WVE': 'WVE (Psion Series 3)',
 'XI': 'XI (FastTracker 2)'}

In [5]:
sf.available_subtypes()

{'PCM_S8': 'Signed 8 bit PCM',
 'PCM_16': 'Signed 16 bit PCM',
 'PCM_24': 'Signed 24 bit PCM',
 'PCM_32': 'Signed 32 bit PCM',
 'PCM_U8': 'Unsigned 8 bit PCM',
 'FLOAT': '32 bit float',
 'DOUBLE': '64 bit float',
 'ULAW': 'U-Law',
 'ALAW': 'A-Law',
 'IMA_ADPCM': 'IMA ADPCM',
 'MS_ADPCM': 'Microsoft ADPCM',
 'GSM610': 'GSM 6.10',
 'G721_32': '32kbs G721 ADPCM',
 'G723_24': '24kbs G723 ADPCM',
 'DWVW_12': '12 bit DWVW',
 'DWVW_16': '16 bit DWVW',
 'DWVW_24': '24 bit DWVW',
 'VOX_ADPCM': 'VOX ADPCM',
 'DPCM_16': '16 bit DPCM',
 'DPCM_8': '8 bit DPCM',
 'VORBIS': 'Vorbis',
 'ALAC_16': '16 bit ALAC',
 'ALAC_20': '20 bit ALAC',
 'ALAC_24': '24 bit ALAC',
 'ALAC_32': '32 bit ALAC'}

In [6]:
#data, sr = librosa.load('my_data/flutter_sound_example.pcm', sr=44100, )

with open('my_data/flutter_sound_example.pcm', 'rb') as f:
    data, samplerate = sf.read(f, channels=1, samplerate=44100, subtype='PCM_16', format='RAW')


# Teste de audibilidade
sf.write('my_data/flutter_sound_example.wav', data, 44100)

# Funcionou! Agora vamos ver
pd.DataFrame(data).describe()

,0
count,109536.000000
mean,0.000119
std,0.036661
min,-0.389343
25%,-0.009216
50%,0.000000
75%,0.011292
max,0.247375


In [7]:
# teste reprodução zero-crossing-rate (zcr)

import numpy as np
import pandas as pd
from librosa import util, core

y = np.array([-1,-2,-3,-4,0,6,7,-8,9,9])
frame_length = 6 #2048
hop_length = 2 #512

print('y:',y, len(y))

padding = [(0, 0) for _ in range(y.ndim)]
print('padding:', padding)
padding[-1] = (int(frame_length // 2), int(frame_length // 2))
print('padding:', padding)

y = np.pad(y, padding, mode="edge")
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(len(y))

y_framed = util.frame(y, frame_length=frame_length, hop_length=hop_length)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(pd.DataFrame(y_framed))

kwargs = {}
kwargs["axis"] = -2
kwargs.setdefault("pad", False)

crossings = core.zero_crossings(y_framed, **kwargs)
print(crossings)

print(np.mean(crossings))

y: [-1 -2 -3 -4  0  6  7 -8  9  9] 10
padding: [(0, 0)]
padding: [(3, 3)]
16
   0  1  2  3  4  5
0 -1 -1 -2 -4  6 -8
1 -1 -1 -3  0  7  9
2 -1 -2 -4  6 -8  9
3 -1 -3  0  7  9  9
4 -2 -4  6 -8  9  9
5 -3  0  7  9  9  9
[[False False False False False False]
 [False False False  True False  True]
 [False False False False  True False]
 [False False  True False  True False]
 [False False False  True False False]
 [False  True False  True False False]]
0.2222222222222222


In [8]:
import ast

with open('../app/assets/ae_data/audio_engineering_zcr_feat.txt', 'r') as f:
   zcr = np.array([ast.literal_eval(line) for line in f][0], dtype='float32')

zcr_mean = np.mean(zcr)

print('zcr:', zcr_mean, 'é igual ao valor no flutter:', 0.05282953886310903, '- fechou!')


zcr: 0.052829538 é igual ao valor no flutter: 0.05282953886310903 - fechou!


In [9]:
# teste reprodução root-mean-square (rms)
y = np.array([-1,-2,-3,-4,0,6,7,-8,9,9])
print('y:',y, len(y))

padding = [(0, 0) for _ in range(y.ndim)]
print('padding:', padding)
padding[-1] = (int(frame_length // 2), int(frame_length // 2))
print('padding:', padding)

y = np.pad(y, padding, mode="constant")
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(pd.DataFrame(y_framed))
print(len(y))

y_framed = util.frame(y, frame_length=frame_length, hop_length=hop_length)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(pd.DataFrame(y_framed))

sum_y = np.sum(np.abs(y_framed) ** 2, axis=-2, keepdims=True)
print('sum', sum_y)

 # Calculate power
power = np.mean(np.abs(y_framed) ** 2, axis=-2, keepdims=True)
print(power)

sqrt_val = np.sqrt(power)
print(sqrt_val)

print(np.mean(sqrt_val)) # 4.643745159797384 fechou com flutter


y: [-1 -2 -3 -4  0  6  7 -8  9  9] 10
padding: [(0, 0)]
padding: [(3, 3)]
16
   0  1  2  3  4  5
0  0  0 -2 -4  6 -8
1  0 -1 -3  0  7  9
2  0 -2 -4  6 -8  9
3 -1 -3  0  7  9  0
4 -2 -4  6 -8  9  0
5 -3  0  7  9  0  0
sum [[ 14  30 114 246 311 226]]
[[ 2.33333333  5.         19.         41.         51.83333333 37.66666667]]
[[1.52752523 2.23606798 4.35889894 6.40312424 7.19953702 6.13731755]]
4.643745159797384


In [10]:
import ast

with open('../app/assets/ae_data/audio_engineering_rmse_feat.txt', 'r') as f:
   rmse = np.array([ast.literal_eval(line) for line in f][0], dtype='float32')

rmse_mean = np.mean(rmse)

print('rms:', rmse_mean, 'é igual ao valor no flutter:', 0.11672754127981479, '- fechou!')


rms: 0.11672754 é igual ao valor no flutter: 0.11672754127981479 - fechou!


In [11]:
from librosa import core
import numpy as np

# teste reprodução spectral_centroid
sr=44100
n_fft=6 # mesmo que frame_length???
#n_fft=2048 # mesmo que frame_length???

freq=None
win_length=n_fft #None
window="hann"
center=True
pad_mode="constant"

y = np.array([-1,-2,-3,-4,0,6,7,-8,9,9], dtype='float32')
print('y:', y)

S, n_fft = core.spectrum._spectrogram(
    y=y,
    n_fft=n_fft,
    hop_length=hop_length,
    win_length=win_length,
    window=window,
    center=center,
    pad_mode=pad_mode,
)
print('n_fft:', n_fft)
print('S:', S)

fft_window = core.spectrum.get_window(window, win_length, fftbins=True)
print('fft_window1:', fft_window, window)
# Pad the window out to n_fft size
fft_window = util.pad_center(fft_window, size=n_fft)
print('fft_window2:', fft_window)
# Reshape so that the window can be broadcast
fft_window = util.expand_to(fft_window, ndim=1 + y.ndim, axes=-2)
print('fft_window3:', fft_window)

padding = [(0, 0) for _ in range(y.ndim)]
padding[-1] = (int(n_fft // 2), int(n_fft // 2))
y = np.pad(y, padding, mode=pad_mode)
print('y_padded:', y)

y_frames = util.frame(y, frame_length=n_fft, hop_length=hop_length)
print('y_frames:', y_frames)

dtype = util.dtype_r2c(y.dtype)
# Pre-allocate the STFT matrix
shape = list(y_frames.shape)
shape[-2] = 1 + n_fft // 2
print('shape:', shape)
stft_matrix = np.empty(shape, dtype=dtype, order="F")

n_columns = util.MAX_MEM_BLOCK // (
    np.prod(stft_matrix.shape[:-1]) * stft_matrix.itemsize
)
n_columns = max(n_columns, 1)
print('n_columns:', n_columns)

fft = core.spectrum.get_fftlib()
print('range:', range(0, stft_matrix.shape[-1], n_columns))
for bl_s in range(0, stft_matrix.shape[-1], n_columns):
    bl_t = min(bl_s + n_columns, stft_matrix.shape[-1])
    print('bl_t:', bl_t, 'bl_s:', bl_s)
    stft_matrix_input = fft_window * y_frames[..., bl_s:bl_t]

    stft_matrix[..., bl_s:bl_t] = fft.rfft(
        stft_matrix_input, axis=-2
    )

    print('stft_matrix_input:', stft_matrix_input)

    print('stft_matrix:', stft_matrix)


print('abs(pow(stft_matrix))', np.abs(stft_matrix) ** 1)

if not np.isrealobj(S):
    raise "Spectral centroid is only defined " "with real-valued input"
elif np.any(S < 0):
    raise "Spectral centroid is only defined " "with non-negative energies"

# Compute the center frequencies of each bin
if freq is None:
    freq = core.convert.fft_frequencies(sr=sr, n_fft=n_fft)
    print('freq:', freq)

if freq.ndim == 1:
    # reshape for broadcasting
    freq = util.expand_to(freq, ndim=S.ndim, axes=-2)
    print('freq2:', freq)



# Column-normalize S
S_norm = util.normalize(S, norm=1, axis=-2)
print('S_norm:', S_norm)

sc = np.sum(freq * S_norm, axis=-2, keepdims=True)
print('sc:', sc)

[0.0, 0.3454915028125263, 0.9045084971874737, 0.9045084971874737, 0.3454915028125263, 0.0]


y: [-1. -2. -3. -4.  0.  6.  7. -8.  9.  9.]
n_fft: 6
S: [[ 3.25       7.75       2.5        7.75      11.5        9.       ]
 [ 2.3848479  5.238082   8.663862   8.792752  12.767145   8.112491 ]
 [ 0.6614378  1.6393596  4.5069394 12.876821  14.756354   5.9529405]
 [ 0.25       1.25       0.5       10.75      10.         4.5      ]]
fft_window1: [0.   0.25 0.75 1.   0.75 0.25] hann
fft_window2: [0.   0.25 0.75 1.   0.75 0.25]
fft_window3: [[0.  ]
 [0.25]
 [0.75]
 [1.  ]
 [0.75]
 [0.25]]
y_padded: [ 0.  0.  0. -1. -2. -3. -4.  0.  6.  7. -8.  9.  9.  0.  0.  0.]
y_frames: [[ 0.  0. -2. -4.  6. -8.]
 [ 0. -1. -3.  0.  7.  9.]
 [ 0. -2. -4.  6. -8.  9.]
 [-1. -3.  0.  7.  9.  0.]
 [-2. -4.  6. -8.  9.  0.]
 [-3.  0.  7.  9.  0.  0.]]
shape: [4, 6]
n_columns: 8192
range: range(0, 6, 8192)
bl_t: 6 bl_s: 0
stft_matrix_input: [[ 0.    0.   -0.   -0.    0.   -0.  ]
 [ 0.   -0.25 -0.75  0.    1.75  2.25]
 [ 0.   -1.5  -3.    4.5  -6.    6.75]
 [-1.   -3.    0.    7.    9.    0.  ]
 [-1.5  -3.   

[0.0,
 0.3454915028125263,
 0.9045084971874737,
 0.9045084971874737,
 0.3454915028125263,
 0.0]

In [12]:
from scipy import signal
window = signal.windows.hann(6)
print('s6:', window)
window = signal.windows.hann(7)
print('s7:', window)

from librosa import core
window = core.spectrum.get_window('hann', 6, fftbins=True)
print('l6:', window)
window = core.spectrum.get_window('hann', 7, fftbins=True)
print('l7:', window)

librosa.show_versions()


s6: [0.        0.3454915 0.9045085 0.9045085 0.3454915 0.       ]
s7: [0.   0.25 0.75 1.   0.75 0.25 0.  ]
l6: [0.   0.25 0.75 1.   0.75 0.25]
l7: [0.         0.1882551  0.61126047 0.95048443 0.95048443 0.61126047
 0.1882551 ]
INSTALLED VERSIONS
------------------
python: 3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)]

librosa: 0.9.1

audioread: 2.1.9
numpy: 1.21.5
scipy: 1.7.3
sklearn: 1.0.2
joblib: 1.1.0
decorator: 5.1.1
soundfile: 0.10.3
resampy: 0.2.2
numba: 0.55.1

numpydoc: None
sphinx: None
sphinx_rtd_theme: None
sphinxcontrib.versioning: None
sphinx-gallery: None
pytest: None
pytest-mpl: None
pytest-cov: None
matplotlib: 3.5.1
presets: None


In [15]:
# https://numpy.org/doc/stable/reference/routines.fft.html#implementation-details
def Ak(x):
    i = 1j
    n = len(x)
    #for k in range(n):
    for k in range(n//2+1):
        sumAm = 0.0j
        for m in range(n):
            #sumAm += x[m] * np.exp(-2 * i * np.pi * m * k / n)
            index = -2 * np.pi * m * k / n
            cmplx = complex(np.cos(index), np.sin(index))
            sumAm += x[m] * cmplx
            #sumAm += x[m] * (np.cos(np.pi * m * k / n) + -2 * np.sin(np.pi * m * k / n))

        yield sumAm

def DFT_slow_mine(x):
    return list(Ak(x))

#https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/
#https://towardsdatascience.com/fast-fourier-transform-937926e591cb#:~:text=As%20the%20name%20implies%2C%20the,)%20to%20O(NlogN)%20.
def DFT_slow(x):
    """Compute the discrete Fourier Transform of the 1D array x"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)


print('their', np.abs(DFT_slow(stft_matrix_input)))
print('mine', DFT_slow_mine(stft_matrix_input))
print('mine_abs', np.abs(DFT_slow_mine(stft_matrix_input)))


print(stft_matrix[1][0], type(stft_matrix[1][0]))

# stft_matrix: [[ -3.25  +0.j         -7.75  +0.j          2.5   +0.j
#     7.75  +0.j         11.5   +0.j          9.    +0.j       ]
#  [  1.375 -1.9485571j   5.125 -1.0825318j  -0.25  +8.6602545j
#    -5.125 -7.1447096j  -8.5   +9.526279j   -2.25  -7.7942286j]
#  [  0.125 +0.649519j   -0.625 +1.5155444j  -1.25  -4.3301272j
#     6.625+11.041824j    7.75 -12.557368j   -4.5   +3.8971143j]
#  [  0.25  +0.j         -1.25  +0.j          0.5   +0.j
#   -10.75  +0.j        -10.    +0.j          4.5   +0.j       ]]

# y: [-1. -2. -3. -4.  0.  6.  7. -8.  9.  9.]

# y_frames: [[ 0.  0. -2. -4.  6. -8.]
#  [ 0. -1. -3.  0.  7.  9.]
#  [ 0. -2. -4.  6. -8.  9.]
#  [-1. -3.  0.  7.  9.  0.]
#  [-2. -4.  6. -8.  9.  0.]
#  [-3.  0.  7.  9.  0.  0.]]

# stft_matrix_input: [[ 0.    0.   -0.   -0.    0.   -0.  ]
#  [ 0.   -0.25 -0.75  0.    1.75  2.25]
#  [ 0.   -1.5  -3.    4.5  -6.    6.75]
#  [-1.   -3.    0.    7.    9.    0.  ]
#  [-1.5  -3.    4.5  -6.    6.75  0.  ]
#  [-0.75  0.    1.75  2.25  0.    0.  ]]

# S: [[ 3.25       7.75       2.5        7.75      11.5        9.       ]
#  [ 2.3848479  5.238082   8.663862   8.792752  12.767145   8.112491 ]
#  [ 0.6614378  1.6393596  4.5069394 12.876821  14.756354   5.9529405]
#  [ 0.25       1.25       0.5       10.75      10.         4.5      ]]

their [[ 3.25        7.75        2.5         7.75       11.5         9.        ]
 [ 2.384848    5.23808171  8.66386173  8.7927527  12.76714533  8.11249037]
 [ 0.66143783  1.63935963  4.50693909 12.87682026 14.75635456  5.95294045]
 [ 0.25        1.25        0.5        10.75       10.          4.5       ]
 [ 0.66143783  1.63935963  4.50693909 12.87682026 14.75635456  5.95294045]
 [ 2.384848    5.23808171  8.66386173  8.7927527  12.76714533  8.11249037]]
mine [array([-3.25+0.j, -7.75+0.j,  2.5 +0.j,  7.75+0.j, 11.5 +0.j,  9.  +0.j]), array([ 1.375-1.94855716j,  5.125-1.08253175j, -0.25 +8.66025404j,
       -5.125-7.14470958j, -8.5  +9.52627944j, -2.25 -7.79422863j]), array([ 0.125 +0.64951905j, -0.625 +1.51554446j, -1.25  -4.33012702j,
        6.625+11.0418239j ,  7.75 -12.55736835j, -4.5   +3.89711432j]), array([  0.25+1.42411614e-15j,  -1.25-7.04171910e-16j,
         0.5 -2.61876575e-15j, -10.75-9.78325901e-15j,
       -10.  -1.68388935e-15j,   4.5 +1.37772765e-15j])]
mine_abs [[ 3.25 

In [36]:
# Big test
#fft: len=2099200; avg=0.09876724660605232

with open('../app/assets/ae_data/audio_engineering_audio_data.txt', 'r') as f:
   audio_data = np.array([ast.literal_eval(line) for line in f][0], dtype='float32')

frame_length = 2048
hop_length = 512

S, n_fft = core.spectrum._spectrogram(
    y=audio_data,
    n_fft=frame_length,
    hop_length=hop_length,
    win_length=frame_length,
    window='hann',
    center=center,
    pad_mode=pad_mode,
)

S_flat = S.flatten()
print('n_fft:', n_fft, 'S len:', len(S_flat), 'data len:', len(audio_data))
S_pd = pd.DataFrame(S_flat)

S_pd.describe().round(2)

n_fft: 2048 S len: 441775 data len: 220500


,0
count,441775.00
mean,0.46
std,4.37
min,0.00
25%,0.00
50%,0.00
75%,0.05
max,282.70


In [2]:
import soundfile as sf

file_name = 'my_data/pcm_data/flutter_sound_realtime_2022-07-14T01_40_30_278476.pcm_stopped'

with open(file_name, 'rb') as f:
    audio_data, sr = sf.read(f, channels=1, samplerate=44100, subtype='PCM_16', format='RAW') #, start=0, frames=44100*5)

sf.write('my_data/pcm_data/test_flutter_sound_realtime.wav', audio_data, sr)

In [17]:
sr = 44100
frame_duration = sr * 5 #s
file_name = 'my_data/pcm_data/flutter_sound_realtime_2022-07-11T21_15_05_554433.pcm_stopped'
tick=3
with sf.SoundFile(file_name, mode='rb', channels=1, samplerate=sr, subtype='PCM_16', format='RAW') as sf_desc:
    sf_desc.seek(int(tick * sr))
    audio_data = sf_desc.read(frames=frame_duration, dtype=np.float32, always_2d=False).T

TypeError: samplerate must be specified